In [49]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [50]:
df_train = pd.read_csv('aggregated_train_NumMosquitos.csv')
df_test = pd.read_csv('aggregated_test.csv')

In [51]:
df_train.columns,df_test.columns

(Index(['Unnamed: 0', 'Species', 'Block', 'Street', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy', 'month', 'day', 'Lat_int', 'Long_int', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x', 'Depth_x', 'SnowFall_x', 'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x', 'ResultSpeed_x', 'ResultDir_x', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y', 'Tavg_y', 'DewPoint_y', 'WetBulb_y', 'Heat_y', 'Cool_y', 'PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y', 'ResultSpeed_y', 'ResultDir_y', 'AvgSpeed_y', 'WnvPresent', 'NumMosquitos'], dtype='object'),
 Index(['Species', 'Block', 'Street', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy', 'month', 'day', 'Lat_int', 'Long_int', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x', 'Depth_x', 'SnowFall_x', 'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x', 'ResultSpeed_x', 'ResultDir_x', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y', 'Tavg_y', 'DewPoint_y',

In [52]:
[var for var in df_train.columns if var not in df_test.columns]

['Unnamed: 0', 'WnvPresent', 'NumMosquitos']

In [57]:
X = df_train
X = X.drop(['NumMosquitos','Unnamed: 0','WnvPresent'],axis=1)
y = df_train['NumMosquitos']


In [58]:
clf = RandomForestRegressor(n_estimators=100)

In [59]:
clf.fit(X,y)

RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0)

In [60]:
yhat = clf.predict(df_test)

In [61]:
y.mean(),yhat.mean()

(12.853512278697886, 6.9815729129195692)

In [62]:
clf.score(df_test,yhat)

1.0

In [63]:
df_test['NumMosquitos'] = yhat

In [66]:
df_test.to_csv('aggregated_test_NumMosquito.csv')

In [65]:
df_test.head()

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,Lat_int,...,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,NumMosquitos
0,2,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,18.268333
1,3,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,17.746500
2,1,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,13.918333
3,4,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,17.376500
4,6,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,16.566500
